In [1]:
import pandas as pd

df = pd.read_csv('data/train_data_categories.csv')
df = df[df['video_id'] != 'b4d70f82038d1d97f1b3ce2a493d12c8'].reset_index(drop=True)

def get_first_level(x):
    tags = []
    for part in x.split(','):
        # tags.append(part.split(':')[0].strip())
        tags.append(part.strip())
    return ', '.join(tags)

def get_second_level(x):
    if x.count(':') > 0:
        return x.split(':')[1].strip()
    else:
        return '-'
    
def get_third_level(x):
    if x.count(':') == 2:
        return x.split(':')[2].strip()
    else:
        return '-'

df['first_level'] = df['tags'].apply(get_first_level)
df['second_level'] = df['tags'].apply(get_second_level)
df['third_level'] = df['tags'].apply(get_third_level)

In [2]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
import re
from tqdm.notebook import tqdm
import os
import json

prompt_template = PromptTemplate(
    input_variables=["title"],
    template=(
        "Классифицируй к какой категории относится заголовок, верни только подходяющую категорию (может быть 1 или несколько). "
        "Напиши только подходящую(ие) категории через ', ' "
        "Транспорт, Книги и литература, Бизнес и финансы, Карьера, Образование, События и достопримечательности, Семья и отношения, Изобразительное искусство, Еда и напитки, Здоровый образ жизни, Хобби и интересы, Дом и сад, Медицина, Фильмы и анимация, Музыка и аудио, Новости и политика, Личные финансы, Животные, Массовая культура, Недвижимость, Религия и духовность, Наука, Покупки, Спорт, Стиль и красота, Информационные технологии, Телевидение, Путешествия, Игры. "
        "Заголовок видео: \"{title}\". "
    ),
)


llm = Ollama(model="llama3", temperature=0)

def process_title_with_llama(title):
    prompt = prompt_template.format(title=title)
    response = llm.invoke(prompt)
    print(response)
    return response

# df = df.iloc[:30]

title_tags = []
for x in tqdm(df['title'].tolist()):
    title_tags.append(process_title_with_llama(x))

df['llm_title_tags'] = title_tags # df['title'].apply(process_title_with_llama)

df['llm_title_tags_list'] = df['llm_title_tags'].apply(lambda x: x.split(', '))
df_llm = df['llm_title_tags_list'].str.join('|').str.get_dummies()

  0%|          | 0/1048 [00:00<?, ?it/s]

Транспорт, Массовая культура
Транспорт, Фильмы и анимация, Массовая культура
Массовая культура, Фильмы и анимация
Транспорт, Спорт
Музыка и аудио
Массовая культура, Музыка и аудио
Категории: Карьера, Образование, Социальные вопросы.
Медицина, Массовая культура
Транспорт, События и достопримечательности, Путешествия
Карьера, Массовая культура
Транспорт, Спорт
Музыка и аудио
Медицина, Массовая культура
Транспорт, Путешествия, Спорт
Категории: Карьера, События и достопримечательности, Массовая культура
Музыка и аудио, Массовая культура
Категории: Карьера, Массовая культура
События и достопримечательности, Спорт
События и достопримечательности, Спорт
Еда и напитки, Кулинария
Карьера, Массовая культура
Категории: Карьера, Образование, Путешествия
Искусство и культура, Изобразительное искусство
Массовая культура, Арт и развлекательные программы
Путешествия
Искусство и культура, Изобразительное искусство
События и достопримечательности, Культура, История
Транспорт, Путешествия
Искусство и кул

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import re
import pandas as pd
import numpy as np

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text

# Разделение классов в first_level на списки
df['first_level_list'] = df['first_level'].apply(lambda x: x.split(', '))

# Преобразование текстов меток в мультилейбел формат
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['first_level_list'])

# Очистка текстов
df['cleaned_title'] = df['title'].apply(clean_text)
df['cleaned_description'] = df['description'].apply(clean_text)

# Разделение данных на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)

X_train_title = train_df['cleaned_title']
X_train_description = train_df['cleaned_description']
llm_train = df_llm.loc[train_df.index]
y_train = y[train_df.index]

X_test_title = test_df['cleaned_title']
X_test_description = test_df['cleaned_description']
llm_test = df_llm.loc[test_df.index]
y_test = y[test_df.index]

tfidf_title = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=350,
    min_df=5,
)
X_train_title = tfidf_title.fit_transform(X_train_title).toarray()
X_test_title = tfidf_title.transform(X_test_title).toarray()

tfidf_description = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=150,
    min_df=5,
)
X_train_description = tfidf_description.fit_transform(X_train_description).toarray()
X_test_description = tfidf_description.transform(X_test_description).toarray()

train_features = np.hstack((X_train_title, llm_train.values))
test_features = np.hstack((X_test_title, llm_test.values))

# Создание модели CatBoostClassifier
catboost_model = CatBoostClassifier(
    iterations=2000, 
    learning_rate=0.03,
    depth=4,
    verbose=500,
    random_seed=42,
    eval_metric='AUC',
    early_stopping_rounds=2000,
)

# Функция для обучения OneVsRestClassifier с CatBoost и eval_set
def fit_ovr_with_eval_set(X_train, y_train, X_valid, y_valid):
    n_classes = y_train.shape[1]
    estimators = []
    
    for i in range(n_classes):
        if y_train.sum(axis=0)[i] < 3 or y_test.sum(axis=0)[i] == 0:
            estimators.append(None)
            continue

        print(f"Training model for class {i + 1}/{n_classes}")
        model = CatBoostClassifier(
            iterations=3500, 
            learning_rate=0.03,
            depth=4,
            verbose=100,
            random_seed=42,
            # eval_metric='AUC',
            early_stopping_rounds=2000,
        )
        model.fit(
            X_train, y_train[:, i],
            eval_set=(X_valid, y_valid[:, i])
        )
        estimators.append(model)
    
    return estimators

# Обучение модели
ovr_estimators = fit_ovr_with_eval_set(train_features, y_train, test_features, y_test)

# Предсказание на тестовых данных
y_pred = np.zeros(y_test.shape)

for i, model in enumerate(ovr_estimators):
    if model is not None:
        y_pred[:, i] = model.predict_proba(test_features)[:, 1]


y_pred = (y_pred > 0.3).astype(np.uint8)

# Вычисление accuracy и F1-меры для мультилейбел данных
accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 micro: {f1_micro:.4f}')
print(f'F1 macro: {f1_macro:.4f}')

Training model for class 8/155
0:	learn: 0.6437704	test: 0.6431403	best: 0.6431403 (0)	total: 49.4ms	remaining: 2m 53s
100:	learn: 0.0016571	test: 0.0013755	best: 0.0013755 (100)	total: 226ms	remaining: 7.59s
200:	learn: 0.0005169	test: 0.0004458	best: 0.0004458 (200)	total: 273ms	remaining: 4.48s
300:	learn: 0.0003244	test: 0.0002922	best: 0.0002922 (300)	total: 321ms	remaining: 3.41s
400:	learn: 0.0002478	test: 0.0002304	best: 0.0002304 (400)	total: 369ms	remaining: 2.85s
500:	learn: 0.0002021	test: 0.0001938	best: 0.0001938 (500)	total: 416ms	remaining: 2.49s
600:	learn: 0.0001722	test: 0.0001694	best: 0.0001694 (600)	total: 465ms	remaining: 2.24s
700:	learn: 0.0001507	test: 0.0001507	best: 0.0001507 (700)	total: 513ms	remaining: 2.05s
800:	learn: 0.0001339	test: 0.0001365	best: 0.0001365 (800)	total: 560ms	remaining: 1.89s
900:	learn: 0.0001208	test: 0.0001245	best: 0.0001245 (900)	total: 608ms	remaining: 1.75s
1000:	learn: 0.0001094	test: 0.0001146	best: 0.0001146 (1000)	total: 65

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [49]:
# # Предсказание на тестовых данных
# y_pred = np.zeros(y_test.shape)

# for i, model in enumerate(ovr_estimators):
#     if model is not None:
#         y_pred[:, i] = model.predict_proba(test_features)[:, 1]



# for t in range(1, 10):
#     print(t / 10)

#     y_pred = np.zeros(y_test.shape)

#     for i, model in enumerate(ovr_estimators):
#         if model is not None:
#             y_pred[:, i] = model.predict_proba(test_features)[:, 1]
    
#     y_pred = (y_pred > (t / 10)).astype(np.uint8)

#     # Вычисление accuracy и F1-меры для мультилейбел данных
#     accuracy = accuracy_score(y_test, y_pred)
#     f1_micro = f1_score(y_test, y_pred, average='micro')
#     f1_macro = f1_score(y_test, y_pred, average='macro')

#     print(f'Accuracy: {accuracy:.4f}')
#     print(f'F1 micro: {f1_micro:.4f}')
#     print(f'F1 macro: {f1_macro:.4f}')

#     print('-----')

0.1


/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.2381
F1 micro: 0.5629
F1 macro: 0.1401
-----
0.2


/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.4405
F1 micro: 0.6222
F1 macro: 0.1400
-----
0.3


...

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.3262
F1 micro: 0.4773
F1 macro: 0.1012
-----
0.8


/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.2595
F1 micro: 0.3937
F1 macro: 0.0862
-----
0.9
Accuracy: 0.1810
F1 micro: 0.2659
F1 macro: 0.0586
-----


/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.multiclass import OneVsRestClassifier
# from catboost import CatBoostClassifier
# from sklearn.metrics import accuracy_score, f1_score
# import re
# import pandas as pd
# import numpy as np

# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
#     return text

# # Разделение классов в first_level на списки
# df['first_level_list'] = df['first_level'].apply(lambda x: x.split(', '))

# # Преобразование текстов меток в мультилейбел формат
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(df['first_level_list'])

# # Очистка текстов
# df['cleaned_title'] = df['title'].apply(clean_text)
# df['cleaned_description'] = df['description'].apply(clean_text)

# # Разделение данных на обучающую и тестовую выборки
# train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)

# X_train_title = train_df['cleaned_title']
# X_train_description = train_df['cleaned_description']
# llm_train = df_llm.loc[train_df.index]
# y_train = y[train_df.index]

# X_test_title = test_df['cleaned_title']
# X_test_description = test_df['cleaned_description']
# llm_test = df_llm.loc[test_df.index]
# y_test = y[test_df.index]

# tfidf_title = TfidfVectorizer(
#     ngram_range=(1, 3),
#     max_features=350,
#     min_df=5,
# )
# X_train_title = tfidf_title.fit_transform(X_train_title).toarray()
# X_test_title = tfidf_title.transform(X_test_title).toarray()

# tfidf_description = TfidfVectorizer(
#     ngram_range=(1, 3),
#     max_features=150,
#     min_df=5,
# )
# X_train_description = tfidf_description.fit_transform(X_train_description).toarray()
# X_test_description = tfidf_description.transform(X_test_description).toarray()

# train_features = np.hstack((X_train_title, llm_train.values))
# test_features = np.hstack((X_test_title, llm_test.values))

# # Создание модели CatBoostClassifier
# catboost_model = CatBoostClassifier(
#     iterations=2000, 
#     learning_rate=0.03,
#     depth=4,
#     verbose=500,
#     random_seed=42,
#     eval_metric='AUC',
#     early_stopping_rounds=2000,
# )

# # Обертка CatBoostClassifier в OneVsRestClassifier
# ovr_model = OneVsRestClassifier(catboost_model.set_fi)

# # Обучение модели на преобразованных данных
# ovr_model.fit(train_features, y_train)

# # Предсказание на тестовых данных
# y_pred = ovr_model.predict(test_features)

# # Вычисление accuracy и F1-меры для мультилейбел данных
# accuracy = accuracy_score(y_test, y_pred)
# f1_micro = f1_score(y_test, y_pred, average='micro')
# f1_macro = f1_score(y_test, y_pred, average='macro')

# print(f'Accuracy: {accuracy:.4f}')
# print(f'F1 micro: {f1_micro:.4f}')
# print(f'F1 macro: {f1_macro:.4f}')

In [7]:
y_pred_labels = mlb.inverse_transform(y_pred)
#y_pred_formatted = [", ".join(labels) for labels in y_pred_labels]

test_df = df.loc[test_df.index]
test_df['predicted_tags'] = y_pred_labels

In [8]:
import pandas as pd
import argparse
import ast
import numpy as np

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(df, pred_df):
    df["tags_new"] = df["tags"].apply(lambda l: l.split(', '))
    df["tags_split"] = df["tags_new"].apply(lambda l: split_tags(l))

    # pred_df["predicted_tags"] = pred_df["predicted_tags"].apply(ast.literal_eval)
    pred_df["predicted_tags_split"] = pred_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in df.iterrows():
        predicted_tags = pred_df[pred_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

sample_df = pd.read_csv('sample_submission.csv')
sample_df['predicted_tags'] = sample_df['predicted_tags'].apply(lambda x: [x])

final_score = find_iou_for_sample_submission(test_df, sample_df)
print(final_score)


final_score = find_iou_for_sample_submission(test_df, test_df)
print(final_score)

0.07057823129251697
0.5838605442176868


In [9]:
test_df[['tags', 'predicted_tags']].iloc[:30]

,tags,predicted_tags
352,Массовая культура: Юмор и сатира,"(Массовая культура: Юмор и сатира,)"
984,Транспорт: Автомобильная культура,"(Карьера, Личные финансы, Массовая культура)"
876,Религия и духовность: Астрология,()
982,Массовая культура: Юмор и сатира,"(Массовая культура: Юмор и сатира,)"
31,"Семья и отношения: Брак и гражданские союзы, М...","(Массовая культура: Юмор и сатира, Семья и отн..."
448,Еда и напитки: Кулинария,"(Еда и напитки: Кулинария,)"
198,"Спорт: Борьба, Массовая культура","(Массовая культура, Спорт: Борьба)"
424,"Путешествия, События и достопримечательности: ...","(Путешествия,)"
107,Массовая культура: Юмор и сатира,"(Массовая культура: Юмор и сатира,)"
712,"Еда и напитки: Кулинария, Массовая культура","(Еда и напитки: Кулинария,)"


In [6]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from catboost import CatBoostClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.linear_model import LogisticRegression
# import re

# # Функция для очистки текста
# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
#     return text

# # Применение очистки текста к колонке 'title'
# df['cleaned_title'] = df['title'].apply(clean_text)

# # Сплит меток по запятой и удаление лишних пробелов
# df['labels'] = df['first_level'].apply(lambda x: [label.strip() for label in x.split(',')])

# # Преобразование меток в формат, подходящий для многометочного обучения
# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(df['labels'])

# X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'], y, test_size=0.4, random_state=42)

# tfidf_params = {
#     # 'max_df': 0.9,
#     'min_df': 5,
#     'ngram_range': (1,2),
#     'max_features': 500
# }

# catboost_model = CatBoostClassifier(
#     iterations=1000, 
#     learning_rate=0.1,
#     depth=6,
#     verbose=False,
#     random_seed=42,
# )

# log_reg_model = LogisticRegression()


# # Создание пайплайна
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(**tfidf_params)),
#     ('catboost', OneVsRestClassifier(catboost_model))
# ])

# # Обучение модели
# pipeline.fit(X_train, y_train)

# # Предсказание на тестовой выборке
# y_pred = pipeline.predict(X_test)

# # Оценка точности и F1-меры
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='micro')
# print(f'Accuracy: {accuracy:.4f}')
# print(f'F1 Score: {f1:.4f}')

# # Доступ к словарю tfidf и количество слов
# tfidf = pipeline.named_steps['tfidf']
# vocab_size = len(tfidf.vocabulary_)
# print(f'TFIDF Vocabulary Size: {vocab_size}')

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 4 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 6 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 11 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 28 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 36 is pre

Accuracy: 0.5452
F1 Score: 0.6641
TFIDF Vocabulary Size: 241


In [ ]:
import pandas as pd
import argparse
import ast
import numpy as np

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(ast.literal_eval)
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter


if __name__ == '__main__':

    try:
        pred_submission = sample_submission #pd.read_csv(pred_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке решения участника'
    try:
        true_submission = ground_truth #pd.read_csv(true_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке эталонного решения'


    final_score = find_iou_for_sample_submission(pred_submission, true_submission)
    print("FINAL_SCORE", 'число от 0 до 1') #final_score)

In [125]:
for x,y in df['tags'].value_counts().to_dict().items():
    print(x, y)

Массовая культура: Юмор и сатира 224
Религия и духовность: астрология 57
Еда и напитки: Кулинария 41
Массовая культура, Карьера 37
Массовая культура 24
Путешествия, События и достопримечательности: Исторические места и достопримечательности 22
Массовая культура: Юмор и сатира, Семья и отношения 22
Спорт: Рыбалка 20
Дом и сад: Дизайн интерьера 20
Массовая культура, Музыка и аудио 18
Транспорт, Спорт: Автогонки, События и достопримечательности: Спортивные события 17
Еда и напитки: Кулинария, Массовая культура 16
Массовая культура, Спорт 15
Музыка и аудио: Комедия и стендап (Музыка и аудио), События и достопримечательности: Концерты и музыкальные мероприятия 14
Массовая культура: Отношения знаменитостей 13
Религия и духовность: астрология, События и достопримечательности: Комедия и стендап 12
Массовая культура, Хобби и интересы: Декоративно-прикладное искусство 11
Семья и отношения 11
Фильмы и анимация: Семейные и детские фильмы, Фильмы и анимация: Фильмы и анимация  11
Массовая культура,

In [121]:
df[df['first_level'] == 'Массовая культура, Карьера']

,video_id,title,description,tags,first_level,second_level,third_level,cleaned_title,text_for_classification
91,51304cc9ff8e942ce481282b6952dd86,Артмеханика. Концерт группы TIHOTIHO,Концерт группы TIHOTIHO,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,концерт группы tihotiho,Концерт группы TIHOTIHO
196,02b797508b9ffc941887703dfb17d365,Артмеханика. Сезон 2. Выпуск 6. Эволюция веду...,Влад Маленко и Ольга Ершова разбираются в том...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,влад маленко и ольга ершова разбираются в том...,Влад Маленко и Ольга Ершова разбираются в том...
220,034753227cf7772245e3fa97c5274932,"Артмеханика. Игра ""Лицо рекламы"".","Игра ""Лицо рекламы""","Массовая культура, Карьера","Массовая культура, Карьера",-,-,игра лицо рекламы,"Игра ""Лицо рекламы"""
234,13aba71d6501df8db686919affbc9b72,Артмеханика. Интервью. Денис Рогов.,Что такое метавселенная? Где мы можем ощутить ...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что такое метавселенная где мы можем ощутить к...,Что такое метавселенная? Где мы можем ощутить ...
236,63d9eab25494382081b6ad100efa7ad4,Артмеханика. Концерт группы Dabro.,Все самые лучшие хиты группы Dabro.,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,все самые лучшие хиты группы dabro,Все самые лучшие хиты группы Dabro.
258,d42d194ba6c4e674a6a625106a76fb1e,Артмеханика. Интервью. Степан Липгарт.,Что собой представляет современная архитектура...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что собой представляет современная архитектура...,Что собой представляет современная архитектура...
259,343470324af6902808606cf1a7503575,Артмеханика. Интервью с сёстрами Набока.,Как сёстры Набока покорили известных актёров и...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,как сёстры набока покорили известных актёров и...,Как сёстры Набока покорили известных актёров и...
261,7413b5707b2119fc392f0ac00cc2a8a1,Артмеханика. Интервью с Анастасией Моргун и Юр...,Зачем сериал Санта-Барбара вернулся к нам спус...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,зачем сериал сантабарбара вернулся к нам спуст...,Зачем сериал Санта-Барбара вернулся к нам спус...
265,9444666b1f0d17c502e46ecbaad08b67,Артмеханика. Сезон 2. Выпуск 15. Подлинное иск...,В нашей студии стало жарко! “Арт Механика” реш...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,в нашей студии стало жарко арт механика решила...,В нашей студии стало жарко! “Арт Механика” реш...
267,c477b6170b5991c1550032c394d93fd0,Артмеханика. Интервью с Евгением Чесом.,Куда приведет тенденция к тотальной кастомизац...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,куда приведет тенденция к тотальной кастомизац...,Куда приведет тенденция к тотальной кастомизац...


In [12]:
from sklearn.metrics import classification_report

print(classification_report(df['first_level'], df_pred['first_level']))

                                                                                       precision    recall  f1-score   support

                                                                    , Стиль и красота       0.00      0.00      0.00         1
                                                                     Бизнес и финансы       0.00      0.00      0.00         6
                                                                            Дом и сад       0.80      0.18      0.30        22
                                                                        Еда и напитки       0.50      0.56      0.53        64
                                                                             Животные       0.00      0.00      0.00         0
                                                                 Здоровый образ жизни       0.00      0.00      0.00         1
                                                                                 Игры       0.12      0.33    

/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca